<a id='start'></a>
# Reti neurali con Tensor Flow

In questo notebook vengono presentati degli esercizi sulle reti neurali con Tensor Flow.

Provate a svolgere il seguente esercizio:<br>
1) [pp -> H -> ZZ -> 4lepton](#section1)<br>

<a id='section1'></a>
## pp -> H -> ZZ -> 4lepton

Creare una rete neurale per analizzare i dati prodotti dagli studi di pp -> H -> ZZ -> 4lepton nel contesto delle ricerche mono-Higgs da parte della collaborazione CMS. I programmi usano le ntuple ridotte (con il nome dell'albero radice HZZ4LeptonsAnalysisReduced) create con Root.

In [2]:
import uproot
import numpy as np
import pandas as pd
import h5py

In [9]:
treename = 'HZZ4LeptonsAnalysisReduced'
filename ='ntuple_4mu_bkg.root'

upfile = uproot.open(filename)
params = upfile[treename].arrays() # dictionary of NumPy arrays

VARS = [b'f_massjj']
df = pd.DataFrame(params, columns=VARS)

In [10]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

treename = 'HZZ4LeptonsAnalysisReduced'

VARS = [b'f_mass4l', b'f_massjj']
upfile_VV = uproot.open('ntuple_4mu_VV.root')
params_VV = upfile_VV[treename].arrays(VARS)
upfile_BKG = uproot.open('ntuple_4mu_bkg.root')
params_BKG = upfile_BKG[treename].arrays(VARS)

df_VV = pd.DataFrame(params_VV, columns=VARS)
df_BKG = pd.DataFrame(params_BKG, columns=VARS)

# convert the feature names to utf-8 to simplify management
for v in VARS:
    df_VV = df_VV.rename(columns={v: v.decode('utf-8')})
    df_BKG = df_BKG.rename(columns={v: v.decode('utf-8')})
VARS = [v.decode('utf-8') for v in VARS]
    
# cut out undefined variables VARS[0] and VARS[1] > -999
df_VV = df_VV[(df_VV[VARS[0]] > -999) & (df_VV[VARS[1]] > -999)]
df_BKG = df_BKG[(df_BKG[VARS[0]] > -999) & (df_BKG[VARS[1]] > -999)] 

# add isSignal variable
df_VV['isSignal'] = np.ones(len(df_VV)) 
df_BKG['isSignal'] = np.zeros(len(df_BKG))

df_all = pd.concat([df_VV, df_BKG])
df_all

TypeError: iterable of expressions must be strings or name, Interpretation pairs (length-2 tuples), not b'f_mass4l'

Creiamo una rete densa, con un singolo livello nascosto completamente connesso con lo stesso numero di neuroni delle variabili di input.

Possiamo quindi usare la funzione di attivazione `sigmoide` per produrre un'uscita di probabilità nell'intervallo da 0 a 1.

Infine usiamo la funzione di loss `binary_crossentropy` durante l'allenamento, una funzione di perdita standard per problemi di classificazione binaria.

Infine utimmizziamo il modello con l'algoritmo Adam per il grandient descend.

[Clicca qui per tornare all'inizio della pagina](#start)<a id='start'></a>